<a href="https://colab.research.google.com/github/s0ye0nyang/News-Recomendation-System/blob/main/news_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Selenium

!apt-get update # to update ubuntu to correctly run apt install

!apt install chromium-chromedriver

In [ ]:
import torch
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정

chrome_options.add_argument('--no-sandbox')

chrome_options.add_argument('--disable-dev-shm-usage')

driver= webdriver.Chrome('chromedriver',options=chrome_options)
driver.implicitly_wait(3) 
date = 10
news_df = pd.DataFrame()
max_page = 10

for i in range(3,5): # categories
  for page in range(3,max_page):
    for main in range(1,5): 
      for child in range(1,5): # news per page
        URL = f"https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=10{i}#&date=%2000:00:00&page={page}"
        select = [f"#main_content > div > div._persist > div:nth-of-type(1) > div:nth-of-type({main}) > div.cluster_body > ul > li:nth-of-type({child}) > div > a",
                  f'#main_content > div > div:nth-of-type({main}) > div > div.cluster_body > ul > li:nth-of-type({child}) > div > a',
                  f"#section_body > ul:nth-of-type({main}) > li:nth-of-type({child}) > dl > dt:nth-of-type(2) > a"]
        driver.get(URL) 
        html = driver.page_source 
        soup = BeautifulSoup(html, 'html.parser')
        for k in range(3):
          if soup.select(select[k]):
            single_news_link = [a['href'] for a in soup.select(select[k])]
            if single_news_link[0]:
              if single_news_link[0][0] == '/':
                single_news_link[0]="https://news.naver.com"+single_news_link[0]
              driver.get(single_news_link[0])
              html = driver.page_source
              soup = BeautifulSoup(html, 'html.parser')
              if soup.select('#articleBodyContents'):
                news_contents = soup.select('#articleBodyContents')[0].get_text().replace('\n','')
                news_df = news_df.append({'제목':single_news_link, 
                                          '기사':news_contents},ignore_index=True)

display(news_df)



In [ ]:
news_df.to_csv("naver_news4.csv", index=False)